In [ ]:
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import tensorflow as tf

In [ ]:
DATASET_PATH = "../input/btcinusd/BTCUSD_1hr.csv"

In [ ]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

In [ ]:
df=pd.read_csv(DATASET_PATH)
data = df['Close']

In [ ]:
data = data.tolist()

In [ ]:
len(data)

In [ ]:
data = data[10000:20000]

In [ ]:
len(data)

In [ ]:
data = np.array(data)

In [ ]:
print(data)

In [ ]:
print(np.min(data), np.max(data))

In [ ]:
data = (data-np.min(data))/(np.max(data)-np.min(data))

In [ ]:
print(np.min(data), np.max(data))

In [ ]:
plt.plot(range(len(data)),data)
# plt.plot(range(len(train_data)),train_data)

plt.show()

In [ ]:
TRAIN_LEN = 9800
VAL_LEN = 100
# train_data = data[0:TRAIN_LEN]
# val_data = data[TRAIN_LEN:TRAIN_LEN+VAL_LEN]

In [ ]:
def make_data(data,len_train):
    X_train = []
    y_train = []
    X_val = []
    y_val = []
    
    X_total = []
    y_total = []
    
    for i in range(len(data)-10):
        X_total.append(data[i])
        if data[i+10] > data[i]:
            y_total.append([1,0])
        else:
            y_total.append([0,1])
    
    X_train = X_total[:len_train]
    y_train = y_total[:len_train]
    
    X_val = X_total[len_train:]
    y_val = y_total[len_train:]
    
    X_train = np.array(X_train).reshape(-1,1,1)
    y_train = np.array(y_train).reshape(-1,2)
    X_val = np.array(X_val).reshape(-1,1,1)
    y_val = np.array(y_val).reshape(-1,2)


    
    return X_train, y_train, X_val, y_val
    

In [ ]:
X_train, y_train, X_val, y_val = make_data(data,TRAIN_LEN)

In [ ]:
print(type(y_train[0]))

In [ ]:
y_train

In [ ]:
counter = 0

for y in y_train:
    if y[0] == 1:
        counter+=1

print(counter/len(y_train))

In [ ]:
# train_X = train_data[:-1].reshape(-1,1,1)
# train_y = train_data[1:].reshape(-1,1)

In [ ]:
# test_X = data.reshape(-1,1,1)
# test_y = data.reshape(-1,1)

In [ ]:
# val_X = val_data[:-1].reshape(-1,1,1)
# val_y = val_data[1:].reshape(-1,1)

In [ ]:
print(X_train, y_train)

In [ ]:
def create_model(stateful):
    model = Sequential()
    model.add(LSTM(128,
              input_shape=(1, 1),
              batch_size=1,
              stateful=stateful))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    return model

print('Creating Stateful Model...')
model_stateful = create_model(stateful=True)

In [ ]:
epochs = 100

In [ ]:
print('Training')
for i in range(epochs):
    print('Epoch', i + 1, '/', epochs)
    
    model_stateful.fit(X_train,
                       y_train,
                       batch_size=1,
                       epochs=1,
#                        verbose=1,
                       validation_data=(X_val, y_val),
                       shuffle=False)
    model_stateful.reset_states()

In [ ]:
model_stateful.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
model_json = model_stateful.to_json()
with open("model.json", "w") as json_file : 
    json_file.write(model_json)

In [ ]:
# print('Predicting')
# predicted_stateful = model_stateful.predict(x_test, batch_size=batch_size)

# print('Plotting Results')
# plt.subplot(3, 1, 1)
# plt.plot(y_test)
# plt.title('Expected')
# plt.subplot(3, 1, 2)
# # drop the first "tsteps-1" because it is not possible to predict them
# # since the "previous" timesteps to use do not exist
# plt.plot((y_test - predicted_stateful).flatten()[tsteps - 1:])
# plt.title('Stateful: Expected - Predicted')

# plt.show()